In [31]:
import tkinter as tk
from tkinter import simpledialog, messagebox
import random
import json
import os

class Game2048:
    def __init__(self, root):
        self.root = root
        self.root.title("2048 Game")
        self.root.resizable(False, False)
        self.grid_size = 4
        self.tile_size = 100
        self.window_size = self.grid_size * self.tile_size
        self.tiles = [[0] * self.grid_size for _ in range(self.grid_size)]
        self.score = 0

        self.create_GUI()
        self.add_new_tile()
        self.add_new_tile()
        self.update_GUI()

        # Key bindings for movement
        root.bind("<Up>", self.move_up)
        root.bind("<Down>", self.move_down)
        root.bind("<Left>", self.move_left)
        root.bind("<Right>", self.move_right)

    def create_GUI(self):
        # Score display
        self.score_label = tk.Label(self.root, text="Score: 0", font=("Helvetica", 16), bg="white", fg="black")
        self.score_label.grid(row=0, column=0, columnspan=self.grid_size, sticky="n", pady=10)

        # Main game grid
        self.main_frame = tk.Frame(self.root, bg="white", width=self.window_size, height=self.window_size)
        self.main_frame.grid(row=1, column=0, columnspan=self.grid_size, padx=10, pady=10)

        # Tile cells
        self.cells = []
        for i in range(self.grid_size):
            row = []
            for j in range(self.grid_size):
                cell = tk.Frame(self.main_frame, width=self.tile_size, height=self.tile_size, bg="lightgray")
                cell.grid(row=i, column=j, padx=5, pady=5)
                label = tk.Label(cell, text="", font=("Helvetica", 24, "bold"), width=4, height=2, bg="lightgray")
                label.pack(expand=True, fill="both")
                row.append(label)
            self.cells.append(row)

        # Control buttons
        self.save_button = tk.Button(self.root, text="Save Game", command=self.save_game)
        self.save_button.grid(row=2, column=0, pady=10)

        self.load_button = tk.Button(self.root, text="Load Game", command=self.load_game)
        self.load_button.grid(row=2, column=1, pady=10)

        self.reset_button = tk.Button(self.root, text="Reset Game", command=self.reset_game)
        self.reset_button.grid(row=2, column=2, pady=10)

    def add_new_tile(self):
        empty_cells = [(i, j) for i in range(self.grid_size) for j in range(self.grid_size) if self.tiles[i][j] == 0]
        if empty_cells:
            i, j = random.choice(empty_cells)
            self.tiles[i][j] = 4 if random.random() > 0.9 else 2

    def update_GUI(self):
        for i in range(self.grid_size):
            for j in range(self.grid_size):
                value = self.tiles[i][j]
                label = self.cells[i][j]
                label.config(text=str(value) if value != 0 else "", bg=self.get_tile_color(value))
                label.config(font=("Helvetica", self.get_font_size(value), "bold"))
        self.score_label.config(text=f"Score: {self.calculate_score()}")

    def get_tile_color(self, value):
        colors = {
            0: "lightgray", 2: "#eee4da", 4: "#ede0c8", 8: "#f2b179",
            16: "#f59563", 32: "#f67c5f", 64: "#f65e3b", 128: "#edcf72",
            256: "#edcc61", 512: "#edc850", 1024: "#edc53f", 2048: "#edc22e"
        }
        return colors.get(value, "black")

    def get_font_size(self, value):
        if value < 100:
            return 24
        elif value < 1000:
            return 20
        else:
            return 16

    def compress(self):
        changed = False
        new_grid = [[0] * self.grid_size for _ in range(self.grid_size)]
        for i in range(self.grid_size):
            pos = 0
            for j in range(self.grid_size):
                if self.tiles[i][j] != 0:
                    new_grid[i][pos] = self.tiles[i][j]
                    if j != pos:
                        changed = True
                    pos += 1
        self.tiles = new_grid
        return changed

    def merge(self):
        changed = False
        for i in range(self.grid_size):
            for j in range(self.grid_size - 1):
                if self.tiles[i][j] == self.tiles[i][j + 1] and self.tiles[i][j] != 0:
                    self.tiles[i][j] *= 2
                    self.tiles[i][j + 1] = 0
                    changed = True
        return changed

    def move_left(self, event):
        if self.compress() | self.merge() | self.compress():
            self.add_new_tile()
            self.update_GUI()
            self.check_game_over()

    def move_right(self, event):
        self.reverse()
        if self.compress() | self.merge() | self.compress():
            self.reverse()
            self.add_new_tile()
            self.update_GUI()
            self.check_game_over()

    def move_up(self, event):
        self.transpose()
        if self.compress() | self.merge() | self.compress():
            self.transpose()
            self.add_new_tile()
            self.update_GUI()
            self.check_game_over()

    def move_down(self, event):
        self.transpose()
        self.reverse()
        if self.compress() | self.merge() | self.compress():
            self.reverse()
            self.transpose()
            self.add_new_tile()
            self.update_GUI()
            self.check_game_over()

    def reverse(self):
        for i in range(self.grid_size):
            self.tiles[i].reverse()

    def transpose(self):
        self.tiles = [list(row) for row in zip(*self.tiles)]

    def calculate_score(self):
        return sum(sum(row) for row in self.tiles)

    def check_game_over(self):
        for i in range(self.grid_size):
            for j in range(self.grid_size):
                if self.tiles[i][j] == 0:
                    return
                if j < self.grid_size - 1 and self.tiles[i][j] == self.tiles[i][j + 1]:
                    return
                if i < self.grid_size - 1 and self.tiles[i][j] == self.tiles[i + 1][j]:
                    return
        messagebox.showinfo("Game Over", "Game Over!")

    def save_game(self):
        state_name = simpledialog.askstring("Save Game", "Enter a name for this game state:")
        if state_name:
            game_state = {
                "tiles": self.tiles,
                "score": self.calculate_score()
            }
            with open(f"{state_name}.json", "w") as file:
                json.dump(game_state, file)
            messagebox.showinfo("Game Saved", f"Game saved as '{state_name}'.")

    def load_game(self):
        saved_files = [f for f in os.listdir() if f.endswith(".json")]
        if saved_files:
            load_window = tk.Toplevel(self.root)
            load_window.title("Load Game")
            tk.Label(load_window, text="Select a saved game state:").pack(pady=5)

            for file_name in saved_files:
                state_name = file_name.replace(".json", "")
                frame = tk.Frame(load_window)
                frame.pack(pady=2, padx=10, fill="x")

                tk.Button(frame, text=state_name, command=lambda f=file_name: self.load_from_file(f, load_window)).pack(side="left")
                tk.Button(frame, text="Delete", command=lambda f=file_name: self.delete_saved_file(f, frame)).pack(side="right")

        else:
            messagebox.showinfo("Load Game", "No saved game states found.")

    def load_from_file(self, file_name, load_window):
        with open(file_name, "r") as file:
            game_state = json.load(file)
            self.tiles = game_state["tiles"]
            self.score = game_state["score"]
            self.update_GUI()
        load_window.destroy()
        messagebox.showinfo("Game Loaded", f"Loaded game state '{file_name.replace('.json', '')}'.")

    def delete_saved_file(self, file_name, frame):
        os.remove(file_name)
        frame.destroy()
        messagebox.showinfo("Delete State", f"Deleted game state '{file_name.replace('.json', '')}'.")

    def reset_game(self):
        self.tiles = [[0] * self.grid_size for _ in range(self.grid_size)]
        self.score = 0
        self.add_new_tile()
        self.add_new_tile()
        self.update_GUI()

if __name__ == "__main__":
    root = tk.Tk()
    game = Game2048(root)
    root.mainloop()